# Modelo de DEEP LEARNING baseado na memória de curto prazo - LSTM

## 1. Bibliotecas

In [ ]:
import math
import numpy                  as np
import pandas                 as pd
import yfinance               as yf

import matplotlib.pyplot      as plt

from datetime                 import datetime
from datetime                 import timedelta

from sklearn.preprocessing    import MinMaxScaler

from keras.models             import Sequential
from keras.layers             import Dense, LSTM

# pd.options.mode.chained_assigment = None

## 2. Ativo de interesse

In [ ]:
acao = 'MGLU3.SA'

inicio = '2014-12-31'
final  = '2023-05-26'

dados_acao = yf.download(acao, inicio, final)

dados_acao

In [ ]:
# Selecionaremos o valor de fechamento 'Close', ao invés da cotação ajustada 'Adj Close' (mais comunmente encontrada
# nos modelos de financças), pois precisamos de uma cotação real;

cotacao = dados_acao['Close'].to_numpy().reshape(-1, 1) # reescalando os dados com o numpy
cotacao

In [ ]:
tamanho_dados_treinamento = int(len(cotacao) * 0.8)

tamanho_dados_treinamento

## 3. Separando os dados em treino e teste

In [ ]:
# Escalar os dados entre 0 e 1

escalador = MinMaxScaler(feature_range=(0, 1))

dados_entre_0_e_1_treinamento = escalador.fit_transform(cotacao[0:tamanho_dados_treinamento, :])

dados_entre_0_e_1_teste = escalador.transform(cotacao[tamanho_dados_treinamento: , :])

dados_entre_0_e_1 = list(dados_entre_0_e_1_treinamento.reshape(
    len(dados_entre_0_e_1_treinamento))) + list(dados_entre_0_e_1_teste.reshape(len(dados_entre_0_e_1_teste)))

dados_entre_0_e_1 = np.array(dados_entre_0_e_1).reshape(len(dados_entre_0_e_1), 1)

dados_entre_0_e_1

In [ ]:
dados_para_treinamento = dados_entre_0_e_1[0: tamanho_dados_treinamento, :]

# dados usados para gerar o resultado
treinamento_x = []

# cotação real
treinamento_y = []

for i in range(60, len(dados_para_treinamento)):
    
    # 60 últimos dias
    treinamento_x.append(dados_para_treinamento[i - 60: i, 0])
    
    # cotação real
    treinamento_y.append(dados_para_treinamento[i, 0])
    
    if i <= 61:
        print(treinamento_x)
        print(treinamento_y)


In [ ]:
# Transformando as listas em arrays e dando um reshape, pois o modelo necessida dos dados em 3 Dimensões

treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)
treinamento_x

## 4.0 Modelo

In [ ]:
modelo = Sequential()

# Modelo com 50 neurônios;
# return sequences = True pois usaremos outro LSTM depois;
# definir o shape, que no caso são 60 informações para gerar um valor;
# adicionar mais neurônios com o dense de 25 e 1

modelo.add(LSTM(50, return_sequences=True, input_shape = (treinamento_x.shape[1], 1))) # Modelo de 50 neurônios + return_sequences = True para adicionar mais 50 neurônios
modelo.add(LSTM(50, return_sequences=False))
modelo.add(Dense(25))
modelo.add(Dense(1))

treinamento_x.shape[1]

In [ ]:
# Copilando o modelo

# A função de Loss é a forma de medir o ERRO do modelo, que nesse caso é classificado como o Erro Médio Quadrátrico que é
# usado na Regresão Linear

modelo.compile(optimizer = 'adam', loss='mean_squared_error')

In [ ]:
# Agora com o modelo copilado e os dados, treinaremos o modelo
# batch size -> neste caso o modelo irá otimizar os parâmetros a cada novo dado
# epochs -> quantas vezes o algoritmo irá rodar os dados de treinamento

modelo.fit(treinamento_x, treinamento_y, batch_size=1, epochs=1)

### 4.1 Modelo de teste

In [ ]:
dados_teste = dados_entre_0_e_1[tamanho_dados_treinamento - 60:, :]

teste_x = []
teste_y = cotacao[tamanho_dados_treinamento: , :]

for i in range(60, len(dados_teste)):
    teste_x.append(dados_teste[i - 60: i, 0])

In [ ]:
# reshape

teste_x = np.array(teste_x)
teste_x = teste_x.reshape(teste_x.shape[0], teste_x.shape[1], 1)

### 4.2 Predições do Modelo

In [ ]:
predicoes = modelo.predict(teste_x)

# Tirando a escala dos dados
predicoes = escalador.inverse_transform(predicoes)

predicoes

### 4.3 Erro Médio Quadrático

In [ ]:
rmse = np.sqrt(np.mean(predicoes - teste_y) ** 2)
rmse

In [ ]:
# Criando o gráfico do modelo

treinamento = dados_acao.iloc[:tamanho_dados_treinamento, :]
df_teste = pd.DataFrame({'Close': dados_acao['Close'].iloc[tamanho_dados_treinamento:],
                        'predicoes': predicoes.reshape(len(predicoes))})

In [ ]:
plt.figure(figsize = (16, 8))
plt.title('Modelo')
plt.xlabel('Data', fontsize = 20)
plt.ylabel('Preço de fechamento', fontsize = 20)
plt.plot(treinamento['Close'])
plt.plot(df_teste[['Close', 'predicoes']])
plt.legend(['Treinamento', 'Real', 'Predições'], loc = 2, prop={'size' = '16')